## A python version of the night-time lights image.
https://github.com/google/earthengine-api  
13 Apr 2022  
NSh  
This notebook is a quick example of some band math and custom visualization.

In [ ]:
# Import, initialize, and authenticate the GEE API
import ee
import collections
collections.Callable = collections.abc.Callable
ee.Authenticate()

In [ ]:
# Initialize and import packages.
import geemap
ee.Initialize()

In [ ]:
#Compute the trend of night-time lights.

# Add a band containing image date as years since 1991.
def create_time_band(img):
    year = ee.Date(img.get('system:time_start')).get('year').subtract(1991)
    return ee.Image(year).byte().addBands(img)


In [ ]:
# Map the time band creation helper over the night-time lights collection.
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_DMSP-OLS_NIGHTTIME_LIGHTS
collection = (ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS')
    .select('stable_lights')
    .map(create_time_band))


In [ ]:
# Compute a linear fit over the series of values at each pixel, 
# visualizing the y-intercept in green, and positive/negative slopes as red/blue.
# Create a map centered at (lat, lon).
Map = geemap.Map(center=[50, 5], zoom=4)
Map.addLayer(
    collection.reduce(ee.Reducer.linearFit()),
    {'min': 0, 'max': [0.18, 20, -0.18], 'bands': ['scale', 'offset', 'scale']},
    'stable lights trend')

# Display map
Map

